In [ ]:
## Estevan R. Tomazini - PUCPR
## Depertamento de TI
## Curso de BigData e inteligência analítica
## Atividade foi feita até uma parte com Jéssica Ferreira
 

In [ ]:
## Será feito o DF sobre basket pertencente a tribo dos analogizers
## modelo de KNN (K - nearest neighbor) que busca por similaridades 
## primeira etapa é a importação das bibliotecas que a principio serão utilizadas

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # faz perfil de database
import sklearn
from sklearn.model_selection import train_test_split # utiliza split entre os testes
from sklearn.neighbors import KNeighborsRegressor # KNN method
from sklearn.ensemble import RandomForestRegressor # RandomForrest method
from sklearn.preprocessing import RobustScaler # todas as entradas estarão na mesma escala numerica

In [ ]:
## leitura do DF a partir do método read.csv() da biblioteca pandas
## como foi feito no google colab, por razões do voilà ter acabado com meu Jupyter lab
## o arquivo foi guardado na memória de trabalho da maquina

In [ ]:
df = pd.read_csv('/nba_stats.csv')

In [ ]:
## A seguir será mostrada a estrutura do DF sendo anaizado,
## google colab permite a visualização sem a necessidade da função print
## ou mesmo de métodos de visualização do tipo head(), tail(), info() e afins
## apenas utilizando a 'magic point'.

In [ ]:
df


,Ano,Posicao,Idade,Jogos,TrueShootingPercentage,FreeThrowRate,OffensiveWinShares,DefensiveWinShares,WinShares,FieldGoals,...,TwoPointFieldGoals,TwoPointFieldGoalAttempts,TwoPointFieldGoalPercentage,EffectiveFieldGoalPercentage,FreeThrows,FreeThrowAttempts,FreeThrowPercentage,Assists,PersonalFouls,Points
0,1950,G-F,31.0,63,0.368,0.467,-0.1,3.6,3.5,144,...,144,516,0.279,0.279,170,241,0.705,176,217,458
1,1950,SG,29.0,49,0.435,0.387,1.6,0.6,2.2,102,...,102,274,0.372,0.372,75,106,0.708,109,99,279
2,1950,SF,25.0,67,0.394,0.259,0.9,2.8,3.6,174,...,174,499,0.349,0.349,90,129,0.698,140,192,438
3,1950,F,24.0,15,0.312,0.395,-0.5,-0.1,-0.6,22,...,22,86,0.256,0.256,19,34,0.559,20,29,63
4,1950,F,24.0,13,0.308,0.378,-0.5,-0.1,-0.6,21,...,21,82,0.256,0.256,17,31,0.548,20,27,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24619,2017,PF,24.0,62,0.604,0.442,3.4,2.2,5.6,253,...,253,442,0.572,0.571,133,196,0.679,99,189,639
24620,2017,C,27.0,51,0.508,0.247,0.5,0.6,1.0,78,...,78,157,0.497,0.494,22,39,0.564,42,61,178
24621,2017,C,20.0,19,0.346,0.161,-0.1,0.1,0.0,10,...,10,31,0.323,0.323,3,5,0.600,4,17,23
24622,2017,SF,22.0,44,0.503,0.181,-0.3,0.8,0.5,88,...,55,122,0.451,0.473,31,40,0.775,36,78,240


In [ ]:
## Para demonstrar a diferença com o método .head(),
## Como a ferramenta do google colab conta com apresentação de dados
## mais harmõnica

In [ ]:
df.head



<bound method NDFrame.head of         Ano Posicao  Idade  Jogos  TrueShootingPercentage  FreeThrowRate  \
0      1950     G-F   31.0     63                   0.368          0.467   
1      1950      SG   29.0     49                   0.435          0.387   
2      1950      SF   25.0     67                   0.394          0.259   
3      1950       F   24.0     15                   0.312          0.395   
4      1950       F   24.0     13                   0.308          0.378   
...     ...     ...    ...    ...                     ...            ...   
24619  2017      PF   24.0     62                   0.604          0.442   
24620  2017       C   27.0     51                   0.508          0.247   
24621  2017       C   20.0     19                   0.346          0.161   
24622  2017      SF   22.0     44                   0.503          0.181   
24623  2017       C   19.0     38                   0.547          0.206   

       OffensiveWinShares  DefensiveWinShares  WinShares 

In [ ]:
## Mostrado o DataFrame e suas caracteristicas, 
## tive a necessidade de utilizar o método reset_index()
## Observei em discussões que o scikit learn pode gerar problemas de indice,
## foi a forma que encontrei de permitir que o modelo rodasse mais a frente.


df = df.reset_index()

In [ ]:
## outro problema que enfrentei diz respeito a maldição da dimensionalidade.
## As colunas tratadas apresentaram dimensões incompatíveis, por isso quando
## recomeceie, retirei os 'Not a Number' já no início do tratamento.

In [ ]:
df1 = df.dropna()

In [ ]:
## Agora que não teremos mais problemas de variaveis infinitas e NaN,
## a partir da visualização é possível retirar colunas que representam 
## variáveis que dificultarão a análise e/ou modelo. 
## como são muitas colunas, escolhi retirar os indices inicialmente. 

In [ ]:
df_tratado = df1.drop(['Ano', 'Posicao', 'FreeThrowRate', 'OffensiveWinShares', 'DefensiveWinShares', 'FreeThrows', 'FreeThrowPercentage', 'Assists', 'EffectiveFieldGoalPercentage', 'TwoPointFieldGoalPercentage', 'FieldGoalPercentage' ], axis=1)

In [ ]:
## a exclusão de colunas visa tratar e diminuir a complexidade dos dados para o 
## desenvolvimento do modelo. Julgo que principalmente a posição e época jogada
## são dados pouco explicativos. Época jogada, embora possa ter correlação com
## a eficiência devido às regras, pode apenas dificultar a classificação
## dos jogadores
## Posição pode ser relevante mas por classificação deve dificultar
## mais do que explicar.

In [ ]:
df_tratado

,index,Idade,Jogos,TrueShootingPercentage,WinShares,FieldGoals,FieldGoalAttempts,TwoPointFieldGoals,TwoPointFieldGoalAttempts,FreeThrowAttempts,PersonalFouls,Points
0,0,31.0,63,0.368,3.5,144,516,144,516,241,217,458
1,1,29.0,49,0.435,2.2,102,274,102,274,106,99,279
2,2,25.0,67,0.394,3.6,174,499,174,499,129,192,438
3,3,24.0,15,0.312,-0.6,22,86,22,86,34,29,63
4,4,24.0,13,0.308,-0.6,21,82,21,82,31,27,59
...,...,...,...,...,...,...,...,...,...,...,...,...
24619,24619,24.0,62,0.604,5.6,253,443,253,442,196,189,639
24620,24620,27.0,51,0.508,1.0,78,158,78,157,39,61,178
24621,24621,20.0,19,0.346,0.0,10,31,10,31,5,17,23
24622,24622,22.0,44,0.503,0.5,88,221,55,122,40,78,240


In [ ]:
## A complexidade já diminuiu de 22 colunas para 12 nesse primeiro tratamento.
## também foram retirados colunas referidas como taxas e aquelas onde a ação
## é puramente estatistica sem que levem a pontuação propriamente dita, como
## 'XXXPercentage' e 'XXXattempts'

## Ainda acredito que seja necessário retirar colunas

In [ ]:
dft2 = df_tratado.drop(['PersonalFouls', 'TwoPointFieldGoalAttempts', 'FreeThrowAttempts', 'FieldGoalAttempts'], axis=1)

In [ ]:
dft2

,index,Idade,Jogos,TrueShootingPercentage,WinShares,FieldGoals,TwoPointFieldGoals,Points
0,0,31.0,63,0.368,3.5,144,144,458
1,1,29.0,49,0.435,2.2,102,102,279
2,2,25.0,67,0.394,3.6,174,174,438
3,3,24.0,15,0.312,-0.6,22,22,63
4,4,24.0,13,0.308,-0.6,21,21,59
...,...,...,...,...,...,...,...,...
24619,24619,24.0,62,0.604,5.6,253,253,639
24620,24620,27.0,51,0.508,1.0,78,78,178
24621,24621,20.0,19,0.346,0.0,10,10,23
24622,24622,22.0,44,0.503,0.5,88,55,240


In [ ]:
## Mostrando os dados que sobraram do tratamento inicial, na próxima
## etapa iremos iniciar a fase de testes:
## A variável que se deseja prever é a porcentagem de arremeços real,
## dessa forma foi retirada e foram criadas as amostras de 
## teste e treino, como 25% da amostra para treino
dft2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23697 entries, 0 to 24623
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   23697 non-null  int64  
 1   Idade                   23697 non-null  float64
 2   Jogos                   23697 non-null  int64  
 3   TrueShootingPercentage  23697 non-null  float64
 4   WinShares               23697 non-null  float64
 5   FieldGoals              23697 non-null  int64  
 6   TwoPointFieldGoals      23697 non-null  int64  
 7   Points                  23697 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 1.6 MB


In [ ]:
## split entre treinamento e teste. Base de testes avalia desempenho do algoritmo. 
## conta com reproducibilidade random_state
## shuffle
## stratify
## X e Y são padrão da bibliotéca, por isso foram mantidos com essa nomenclatura.

X_train_bskt, X_test_bskt, Y_train_bskt, Y_test_bskt = train_test_split(dft2.drop('TrueShootingPercentage', axis = 1), dft2 ['TrueShootingPercentage'], test_size = 0.25, random_state=0)

In [ ]:
## anteriormente tive problemas por ter NaN e dados float64 tidos como infinitos,
## na estrutura de dados, foi corrigido no inicio dessa vez
## por isso foi utilizado o método reset_index() no inicio do tratamento do DF,

print ('teste x nan')
np.any(np.isnan(X_train_bskt))


teste x nan


False

In [ ]:
print ('teste x infinito')
np.all(np.isfinite(X_train_bskt))



teste x infinito


True

In [ ]:
print ('teste y nan')
np.any(np.isnan(Y_train_bskt))


teste y nan


True

In [ ]:
print ('teste y infinito')
np.all(np.isfinite(Y_train_bskt))

teste y infinito


False

In [ ]:
## Agora rodaremos o modelo KNN, que agrupa por valor de ki para 
## regressão/classificação
## visto que a intenção é prever os acertos/jogador baseado
## em dados que levam a pontuação 

In [ ]:
modelo_KNN = KNeighborsRegressor().fit(X_train_bskt, Y_train_bskt) 

In [ ]:
## sabemos que meu modelo preliminar ter baixa eficiencia, 
## com apenas 25% de score, onde o Lim -> infinito leva X -> 1.

In [ ]:
modelo_KNN.score (X_test_bskt, Y_test_bskt) 

0.2572034460112269

In [ ]:
## Dessa forma será necessario revalidar as variáveis
## utilizadas nesse modelo.
## embora não tenha entregado muito nessa etapa,
## ML é a area que pretendo seguir,
## pretendo realizar análises de todas as CSV disponibilizadas
## e melhorar o desempenho da análise.